In [1]:
import cv2
import mediapipe as mp
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import ImageFont, ImageDraw, Image
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:


# 모델 아키텍처 정의
class HandStateNN(nn.Module):
    def __init__(self, n_classes):
        super(HandStateNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(63, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, n_classes)
        )
    def forward(self, x):
      return self.fc(x)

# 학습된 모델 로드
model = HandStateNN(n_classes=1)
model.load_state_dict(torch.load('hand_model.pth', map_location=device)) # 모델 파일 경로 지정
model.eval()

# 한글 폰트 사용 시
# 한글 폰트 경로 설정 
# font_path = 'AppleGothic.ttf'
# font = ImageFont.truetype(font_path, 20)

# MediaPipe 손 모델 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# # 라렝링을 위한 레이블 이름 설정
# labels_map = {
#     0: 'Release',
#     1: 'Folding hand',
#     2: 'Grab'
# }

# 카메라 초기화
cap = cv2.VideoCapture(1) # for Mac
# cap = cv2.VideoCapture(0) # for Windows

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 이미지 전처리
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(frame, 1)
    results = hands.process(image)

    # 랜드마크 추출
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            landmarks = torch.tensor([landmarks]).flatten().unsqueeze(0)
    
            # 모델을 사용하여 라벨 예측
            with torch.no_grad():
                predictions = model(landmarks)
                predicted_label = torch.argmax(predictions, axis=1)
                label_text = str(round(float(predictions.item()), 3))
                # label_text = labels_map[int(predicted_label.item())]
    
                # # 한글 폰트 사용 시
                # # PIL을 사용하여 한글 라벨링
                # img_pil = Image.fromarray(frame)
                # draw = ImageDraw.Draw(img_pil)ㅂ
                # draw.text((10, 30), label_text, font=font, fill=(0, 255, 0))
                # frame = np.array(img_pil)
                
                # 화면에 랜드마크 그리기 및 라벨링 표시
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                cv2.putText(image, label_text, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 3, cv2.LINE_AA)

    # 화면에 결과 보여주기
    cv2.imshow('Hand State Recognition', image)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1) # for Mac

-1